In [22]:
import os
import sys
import openai
from dotenv import dotenv_values
from langchain.chat_models import AzureChatOpenAI
sys.path.append("..") ## add directory above


In [23]:
env_name = "../llm.env" # change to use your own .env file
config = dotenv_values(env_name)

#Azure OpenAI
openai.api_type = config["OPENAI_API_TYPE"] #"azure"
openai.api_key = config['OPENAI_API_KEY']
openai.api_base = config['OPENAI_API_BASE']
openai.api_version = config['OPENAI_API_VERSION']

In [24]:
engine = "gpt-4-32k"
#engine = "gpt-35-turbo"
llm = AzureChatOpenAI(
    deployment_name=engine,
    openai_api_base=openai.api_base,
    openai_api_version=openai.api_version,
    openai_api_key=openai.api_key,
    openai_api_type = openai.api_type,
    temperature=0.0, verbose = True
)

## 1. Demonstrating conversation summary memory chain:
This chain summarizes the previous user conversation and appends the summary to context for answering questions 

In [25]:
from chatbotSkills import qa_chain_ConversationSummaryMemory

# Make a Question Answer chain function and pass 
prefix_template = """
        You are a chatbot having a conversation with a human. 
        Given the Context, Chat History, and a Human Query, 
        create a final answer. Don't hallucinate at all. If you don't have an answer, say "I don't know". """

qa_chain = qa_chain_ConversationSummaryMemory(llm, prefix_template=prefix_template, to_debug=False)

## Question Answering

#Question 1
answer = qa_chain.run({
        'context': "USSA is a space agency in county Y. It is a government agency responsible for the exploration and development of space.",
        'human_input': "What is USSA" 
})

print("Question 1: ")
print(answer)

# Question 2: 
answer = qa_chain.run({
        'context': "Zootopia is a 2016 American computer-animated buddy cop action comedy film produced by Walt Disney Animation Studios.",
        'human_input': "Do you know about any space agency?" 
}) 

print("Question 2: ")
print(answer)
print()
print ("""Context in question 2 does not contain any specific information regarding the 
       user question but still llm provides correct answer by using the memory of previous conversation""")

# print(qa_chain.memory) ## You can see the memory using this call

Question 1: 
USSA is a space agency in county Y. It is a government agency responsible for the exploration and development of space.
Question 2: 
Yes, I mentioned earlier about the USSA, a government space agency in county Y, responsible for space exploration and development.

Context in question 2 does not contain any specific information regarding the 
       user question but still llm provides correct answer by using the memory of previous conversation


## 2. Demonstrating conversation buffer memory chain:
This chain summarizes the previous user conversation and appends the summary to context for answering questions 


In [26]:
from chatbotSkills import qa_chain_ConversationBufferMemory

# Make a Question Answer chain function and pass 
prefix_template = """
        You are a chatbot having a conversation with a human. 
        Given the Context, Chat History, and a Human Query, 
        create a final answer. Don't hallucinate at all. If you don't have an answer, say "I don't know". """

qa_chain = qa_chain_ConversationBufferMemory(llm, prefix_template=prefix_template, to_debug=False)

## Question Answering

#Question 1
answer = qa_chain.run({
        'context': "USSA is a space agency in county Y. It is a government agency responsible for the exploration and development of space.",
        'human_input': "What is USSA" 
})

print("Question 1: ")
print(answer)

# Question 2: 
answer = qa_chain.run({
        'context': "Zootopia is a 2016 American computer-animated buddy cop action comedy film produced by Walt Disney Animation Studios.",
        'human_input': "Do you know about any space agency?" 
}) 

print("Question 2: ")
print(answer)
print()
print ("""Context in question 2 does not contain any specific information regarding the 
       user question but still llm provides correct answer by using the memory of previous conversation""")

# print(qa_chain.memory) ## You can see the memory using this call

Question 1: 
USSA is a space agency in county Y. It is a government agency responsible for the exploration and development of space.
Question 2: 
Yes, I do. For instance, USSA is a space agency in county Y. It is a government agency responsible for the exploration and development of space.

Context in question 2 does not contain any specific information regarding the 
       user question but still llm provides correct answer by using the memory of previous conversation


## 3. Demonstrating usre query based context summarization chain:
Sometimes context can be large and don't fit in a prompt window. So, this chain summarizes context given user query 

In [27]:
from chatbotSkills import user_query_based_context_summarization

# Template 
prefix_template = """
        Write a concise summary of the context so that it includes the details related to the human query. """

context_summary_chain = user_query_based_context_summarization(llm, prefix_template=prefix_template, to_debug=False)

context = """
            USSA is a space agency in county Y. It is a government agency responsible
            for the exploration and development of space.
            Zootopia is a 2016 American computer-animated buddy cop action comedy
            film produced by Walt Disney Animation Studios.
            """

#Question 1
answer = context_summary_chain.run({
        'context': context,
        'human_input': "What is USSA?" 
})


print("Question 1: ")
print(answer)
print()

print ("""This llm extracts only relevant information from the context. """)

# print(qa_chain.memory) ## You can see the memory using this call

Question 1: 
USSA is a government space agency in county Y, responsible for the exploration and development of space.

This llm extracts only relevant information from the context. 


## 3. Demonstrating combine_docs:
Sometimes contexts retrieved from the database can be large and doesn't fit in a prompt. So, this code will extract relevant information from the context given the user query.

In [29]:
from chatbotSkills import combine_docs, count_tokens

context_1 = """
    The United Space Exploration Administration (USSA) stands as county Y's premier governmental space agency, 
    entrusted with the monumental task of spearheading the exploration, investigation, and advancement of the cosmic frontier.
    With a dedicated cadre of brilliant scientists, intrepid astronauts, 
    and cutting-edge technology, USSA pioneers a path to unlock the mysteries of the universe and 
    harness its potential for the betterment of humanity. 
    """
context_2 = """
    Through audacious missions and visionary initiatives, 
    USSA propels the nation to new heights, ensuring that the celestial realm 
    becomes a beacon of knowledge, opportunity, and inspiration for generations to come.
    """

context_list = [context_1, context_2]

input_token_count = count_tokens(context_1+context_2, "gpt-4-32k")
prefix_template = """
        Extract information from the context so that it includes the details related to the human query. """
user_query = "What does USSA tands for?" 
max_input_tokens = 100 ## For demonstration, we are assuming that max tokenfor input context should not exceed 100

output = combine_docs(context_list, llm, to_debug=False, max_tokens=max_input_tokens, 
                      user_query=user_query, prefix_template=prefix_template)

output_token_count = count_tokens(output, "gpt-4-32k")
print("input_token_count: ", input_token_count)
print("output_token_count: ", output_token_count)

print("output: ", output)
print()
print("""This demonstrates that the combine_docs function reduces the tokens for the input""")



input_token_count:  134
output_token_count:  33
output:  USSA stands for United Space Exploration Administration, which is county Y's premier governmental space agency responsible for the exploration, investigation, and advancement of the cosmic frontier.

This demonstrates that the combine_docs function reduces the tokens for the input
